### SatelliteBench

* [Dataset](https://physionet.org/content/multimodal-satellite-data/1.0.0/)

* [Paper](https://arxiv.org/abs/2401.11114)

In [ ]:
%cd ..

### Setup Environment:

In [2]:
import os
import pandas as pd

from src.classifiers import process_labels, split_data
from src.classifiers_base import preprocess_df

from transformers import BertTokenizer

from src.multimodal_data_loader import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_base_cpu_metrics import calculate_memory

In [3]:
PATH = 'datasets/satellitedata/'

In [4]:
text_path = os.path.join(PATH, 'labels.csv')
images_path = os.path.join(PATH, 'images')

## Get data

In [5]:
df = pd.read_csv(text_path)
df.Labels = df.Labels.astype(str)
df

,image_id,text,Labels,split,Labels_4,Labels_3
0,73001_2016-06-12.jpg,An image from city Ibagué taken in date 2016-0...,1,train,3,2
1,76001_2017-06-11.jpg,An image from city Cali taken in date 2017-06-...,1,train,1,1
2,8001_2018-04-15.jpg,An image from city Barranquilla taken in date ...,0,train,1,0
3,23001_2016-05-08.jpg,An image from city Montería taken in date 2016...,0,train,1,1
4,5001_2017-04-30.jpg,An image from city Medellín taken in date 2017...,0,train,1,0
...,...,...,...,...,...,...
1555,50001_2017-03-19.jpg,An image from city Villavicencio taken in date...,0,test,1,0
1556,23001_2017-03-26.jpg,An image from city Montería taken in date 2017...,0,test,0,0
1557,8001_2017-01-22.jpg,An image from city Barranquilla taken in date ...,0,test,0,0
1558,76001_2017-09-10.jpg,An image from city Cali taken in date 2017-09-...,0,test,0,0


## Data Perparation

In [6]:
# Select features and labels vectors
text_columns = 'text'
image_columns = 'image_id'
label_columns = 'Labels'

df = preprocess_df(df, image_columns, images_path)

# Split the data
train_df, test_df = split_data(df)

# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)

100%|██████████| 1560/1560 [00:00<00:00, 1811.12it/s]

Train Shape: (936, 6)
Test Shape: (312, 6)


In [7]:
train_df

,image_id,text,Labels,split,Labels_4,Labels_3
0,datasets/satellitedata/images/73001_2016-06-12...,An image from city Ibagué taken in date 2016-0...,1,train,3,2
1,datasets/satellitedata/images/76001_2017-06-11...,An image from city Cali taken in date 2017-06-...,1,train,1,1
2,datasets/satellitedata/images/8001_2018-04-15.jpg,An image from city Barranquilla taken in date ...,0,train,1,0
3,datasets/satellitedata/images/23001_2016-05-08...,An image from city Montería taken in date 2016...,0,train,1,1
4,datasets/satellitedata/images/5001_2017-04-30.jpg,An image from city Medellín taken in date 2017...,0,train,1,0
...,...,...,...,...,...,...
931,datasets/satellitedata/images/23001_2016-03-06...,An image from city Montería taken in date 2016...,1,train,2,1
932,datasets/satellitedata/images/68001_2017-09-03...,An image from city Bucaramanga taken in date 2...,0,train,0,0
933,datasets/satellitedata/images/41001_2018-08-26...,An image from city Neiva taken in date 2018-08...,0,train,0,0
934,datasets/satellitedata/images/54001_2017-06-18...,An image from city Cúcuta taken in date 2017-0...,0,train,0,0


In [8]:
# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Models

In [10]:
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [11]:
calculate_memory(train_loader, test_loader, output_size)

Early fusion:
Average Memory per Batch in Train: 35.83 MB
Total Memory Usage per Epoch Train: 537.48 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 11.94 MB
Total Memory Usage per Epoch Test: 59.72 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.94 MB

Late fusion:
Average Memory per Batch in Train: 35.83 MB
Total Memory Usage per Epoch Train: 537.48 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 11.94 MB
Total Memory Usage per Epoch Test: 59.72 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.57 MB
